# Parse tables to database

https://nbviewer.jupyter.org/github/FinanceData/OpenDartReader/blob/master/docs/OpenDartReader_reference_manual.ipynb


```
KOSPI
└── Companys
    └── Receipts
        ├── BS: Financial statement(Balance Sheet)
        ├── IS: Income Statement
        ├── CIS: Comprehensive Income Statement
        ├── CF: Cash flow statement
        └── SCE: Statement of Changes in Equity
```

<p align="center">
    <img alt="Alt Text" src="https://g.gravizo.com/svg?digraph%20G%20%7B%0A%20%201%20%5Blabel%3D%22KOSPI%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%202%20%5Blabel%3D%22Company1%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%203%20%5Blabel%3D%22Company2%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%204%20%5Blabel%3D%22...%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%205%20%5Blabel%3D%22Receipt%20No.1%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%206%20%5Blabel%3D%22Receipt%20No.2%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%20%20%20%20%0A%20%207%20%5Blabel%3D%22...%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%208%20%5Blabel%3D%22BS%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%209%20%5Blabel%3D%22IS%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%2010%20%5Blabel%3D%22CIS%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%2011%20%5Blabel%3D%22CF%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%2012%20%5Blabel%3D%22SCE%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%201%20-%3E%202%20-%3E%205%3B%0A%20%201%20-%3E%203%3B%0A%20%201%20-%3E%204%3B%0A%20%202%20-%3E%206%3B%0A%20%202%20-%3E%207%3B%0A%20%206%20-%3E%208%3B%0A%20%206%20-%3E%209%3B%0A%20%206%20-%3E%2010%3B%0A%20%206%20-%3E%2011%3B%0A%20%206%20-%3E%2012%3B%0A%7D" />
</p>
<details>
<summary>How to create graph in markdown?</summary>

```python
from urllib.parse import quote
raw = """digraph G {
  1 [label="KOSPI", fontcolor=black, shape=box];
  2 [label="Company1", fontcolor=black, shape=box];
  3 [label="Company2", fontcolor=black, shape=box];
  4 [label="...", fontcolor=black, shape=box];
  5 [label="Receipt No.1", fontcolor=black, shape=box];
  6 [label="Receipt No.2", fontcolor=black, shape=box];    
  7 [label="...", fontcolor=black, shape=box];
  8 [label="BS", fontcolor=black, shape=box];
  9 [label="IS", fontcolor=black, shape=box];
  10 [label="CIS", fontcolor=black, shape=box];
  11 [label="CF", fontcolor=black, shape=box];
  12 [label="SCE", fontcolor=black, shape=box];
  1 -> 2 -> 5;
  1 -> 3;
  1 -> 4;
  2 -> 6;
  2 -> 7;
  6 -> 8;
  6 -> 9;
  6 -> 10;
  6 -> 11;
  6 -> 12;
}"""
txt = quote(raw)
```
    
copy the text behind https://g.gravizo.com/svg?
</details>

# Finance DataReader

In [1]:
from private.apikey import APIKEY

import pandas as pd
import OpenDartReader
import FinanceDataReader as fdr

dart = OpenDartReader(APIKEY) 

In [2]:
stocks = fdr.StockListing("KOSPI")
stocks = stocks.loc[~stocks["Sector"].isnull(), :]
stocks_syms = stocks["Symbol"].values
stocks = stocks.reset_index(drop=True)

In [3]:
stocks

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,박대현,http://www.ajnet.co.kr,서울특별시
1,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
2,027410,KOSPI,BGF,기타 금융업,지주회사,2014-05-19,12월,홍정국,http://www.bgf.co.kr,서울특별시
3,282330,KOSPI,BGF리테일,종합 소매업,체인화 편의점,2017-12-08,12월,이건준,http://www.bgfretail.com,서울특별시
4,138930,KOSPI,BNK금융지주,기타 금융업,금융지주회사,2011-03-30,12월,김지완,http://www.bnkfg.com,부산광역시
...,...,...,...,...,...,...,...,...,...,...
803,079980,KOSPI,휴비스,화학섬유 제조업,"합성섬유(폴리에스테르원사,원면),재생섬유,폴리에스텔 원사,원면,고상칩 제조,도소매",2012-02-23,12월,신유동,http://www.huvis.com,서울특별시
804,005010,KOSPI,휴스틸,1차 철강 제조업,"강관(배관용,구조용,유정용) 제조,도매",1973-06-29,12월,박훈,http://www.husteel.com,서울특별시
805,069260,KOSPI,휴켐스,기타 화학제품 제조업,"화합물,화학제품 제조",2002-10-07,12월,신진용,http://www.huchems.com,서울특별시
806,000540,KOSPI,흥국화재,보험업,손해보험,1974-12-05,12월,권중원,http://www.insurance.co.kr,서울특별시


In [4]:
import sqlite3
from pathlib import Path

db_path = Path("./private/")
conn = sqlite3.connect(db_path / "kospi.db")
# Samsung Electronic: 005930
# conn = sqlite3.connect(db_path / "samsung_new.db")
c = conn.cursor()

In [5]:
stocks.to_sql("kospi", conn, index=True)

## Company Basic

- corp_name: 정식명칭
- corp_name_eng: 영문명칭
- stock_name: 종목명 또는 약식명칭 
- stock_code: 상장회사인 경우 주식의 종목코드
- ceo_nm: 대표자명
- crop_cls: 법인구분
- jurir_no: 법인등록번호
- bizr_no: 사업자등록번호
- adres: 주소
- hm_url: 홈페이지
- ir_url: IR홈페이지
- phn_no: 전화번호
- fax_no: 팩스번호
- induty_code: 업종코드
- estdt: 설립일
- acc_mt: 결산월

In [6]:
from tqdm.notebook import tqdm

In [7]:
basic_cols = [
    'corp_name', 'corp_name_eng', 'stock_name', 'stock_code', 'ceo_nm', 'corp_cls', 
    'jurir_no', 'bizr_no', 'adres', 'hm_url', 'ir_url', 'phn_no', 'fax_no', 'induty_code', 'est_dt', 'acc_mt'
]
# df_basic = pd.DataFrame([dart.company(s) for s in tqdm(stocks_syms)]).loc[:, basic_cols]
# df_basic.to_sql("company", conn, index=True)

  0%|          | 0/808 [00:00<?, ?it/s]

In [36]:
df_samsung = pd.DataFrame([dart.company("005930")]).loc[:, basic_cols]
df_samsung

,corp_name,corp_name_eng,stock_name,stock_code,ceo_nm,corp_cls,jurir_no,bizr_no,adres,hm_url,ir_url,phn_no,fax_no,induty_code,est_dt,acc_mt
0,삼성전자(주),"SAMSUNG ELECTRONICS CO,.LTD",삼성전자,005930,"김기남, 김현석, 고동진",Y,1301110006246,1248100998,경기도 수원시 영통구 삼성로 129 (매탄동),www.sec.co.kr,,031-200-1114,031-200-7538,264,19690113,12


In [ ]:
# DROP TABLE if exists
# sql = "DROP TABLE company"
# res = c.execute(sql)

# insert into "company" table
df_samsung.to_sql("company", conn, index=True)

In [9]:
sql = "SELECT * FROM company"
res = c.execute(sql)
col_company = list(map(lambda x: x[0], res.description))
for col, value in zip(col_company, res.fetchone()):
    print(f"{col}: {value}")

index: 0
corp_name: AJ네트웍스 주식회사
corp_name_eng: AJ Networks Co.,Ltd.
stock_name: AJ네트웍스
stock_code: 095570
ceo_nm: 박대현
corp_cls: Y
jurir_no: 1101111874654
bizr_no: 2148648586
adres: 서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)
hm_url: www.ajnet.co.kr
ir_url: 
phn_no: 02-6363-9999
fax_no: 02-6240-0888
induty_code: 76320
est_dt: 20000210
acc_mt: 12


## Report

- corp (문자열): 검색대상 회사의 종목코드를 지정합니다. 고유번호, 회사이름도 가능합니다.
- key_word (문자열): 조회 내용 지정, 아래 "key_word 항목"을 참고하십시오 ('증자','배당','자기주식','최대주주','최대주주변동','소액주주','임원','직원','임원개인보수','임원전체보수','개인별보수','타법인출자')
- bsns_year (문자열 혹은 정수값): 사업연도
- reprt_code (문자열): 보고서 코드 ('11013'=1분기보고서, '11012'=반기보고서, '11014'=3분기보고서, '11011'=사업보고서)

반환값 (DataFrame): 조회 결과를 데이터프레임(DataFrame)으로 반환합니다. 데이터프레임의 각 컬럼은 다음과 같습니다.

- rcept_no: 접수번호
- corp_cls: 법인구분 Y(유가), K(코스닥), N(코넥스), E(기타)
- corp_code: 고유번호
- corp_name: 법인명

key_word 항목 지정에 따라 결과 데이터의 컬럼이 달라집니다. '배당' - 배당에 관한 사항

- se: 구분. 유상증자(주주배정), 전환권행사 등
- stock_knd: 주식 종류
- thstrm: 당기
- frmtrm: 전기
- lwfr: 전전기

In [39]:
corp = "005930"
bsns_year = 2020

In [40]:
# ['증자', '배당', '자기주식', '최대주주', '최대주주변동', '소액주주', '임원', '직원', '임원개인보수', '임원전체보수', '개인별보수', '타법인출자']
dart.report(corp, '임원개인보수', bsns_year, reprt_code='11011')

,rcept_no,corp_cls,corp_code,corp_name,nm,ofcps,mendng_totamt,mendng_totamt_ct_incls_mendng
0,20210309000744,Y,00126380,삼성전자,최윤호,이사,"3,028,000,000",-
1,20210309000744,Y,00126380,삼성전자,한종희,이사,"4,183,000,000",-
2,20210309000744,Y,00126380,삼성전자,이상훈,사장,"4,594,000,000",-
3,20210309000744,Y,00126380,삼성전자,김현석,대표이사,"5,457,000,000",-
4,20210309000744,Y,00126380,삼성전자,고동진,대표이사,"6,712,000,000",-
5,20210309000744,Y,00126380,삼성전자,김기남,대표이사,"8,274,000,000",-


## Finstate
- corp (문자열): 검색대상 회사의 종목코드를 지정합니다. 고유번호, 회사이름도 가능합니다.
- bsns_year (문자열 혹은 정수값): 사업연도
- reprt_code (문자열): 보고서 코드 ('11013'=1분기보고서, '11012'=반기보고서, '11014'=3분기보고서, '11011'=사업보고서)

반환값 (DataFrame): 조회 결과를 데이터프레임(DataFrame)으로 반환합니다. 데이터프레임의 각 컬럼은 다음과 같습니다.

- rcept_no: 접수번호
- corp_code: 사업 연도
- stock_code: 종목 코드
- reprt_code: 보고서 코드
- account_nm: 계정명 (예: 자본총계)
- fs_div: 개별/연결구분 ('CFS'=연결재무제표, 'OFS'=재무제표)
- fs_nm: 개별/연결명 ('연결재무제표' 또는 '재무제표')
- sj_div: 재무제표구분 ('BS'=재무상태표, 'IS'=손익계산서)
- sj_nm: 재무제표명 ( '재무상태표' 또는 '손익계산서')
- thstrm_nm: 당기명
- thstrm_dt: 당기일자
- thstrm_amount: 당기금액
- thstrm_add_amount: 당기누적금액
- frmtrm_nm: 전기명
- frmtrm_dt: 전기일자
- frmtrm_amount: 전기금액
- frmtrm_add_amount: 전기누적금액
- bfefrmtrm_nm: 전전기명
- bfefrmtrm_dt: 전전일자
- bfefrmtrm_amount: 전전기금액
- ord: 계정과목 정렬순서

In [41]:
df_summary = dart.finstate(corp, bsns_year, reprt_code="11011")
for col in ["thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]:
    df_summary.loc[:, col] = df_summary.loc[:, col].apply(lambda x: int("".join(x.split(","))))
    
cols = ["fs_div", "sj_div", "account_nm", "thstrm_nm", "thstrm_amount"] # "frmtrm_nm", "frmtrm_amount", "bfefrmtrm_nm", "bfefrmtrm_amount"]
df_summary.loc[:, cols].groupby(["thstrm_nm", "fs_div", "sj_div", "account_nm"]).agg("sum")

thstrm_amount
thstrm_nm fs_div sj_div account_nm                 
제 52 기    CFS    BS     부채총계        102287702000000
                        비유동부채        26683351000000
                        비유동자산       180020139000000
                        유동부채         75604351000000
                        유동자산        198215579000000
                        이익잉여금       271068211000000
                        자본금            897514000000
                        자본총계        275948016000000
                        자산총계        378235718000000
                 IS     당기순이익        26407832000000
                        매출액         236806988000000
                        법인세차감전 순이익   36345117000000
                        영업이익         35993876000000
          OFS    BS     부채총계         46347703000000
                        비유동부채         1934799000000
                        비유동자산       155865878000000
                        유동부채         44412904000000
                        유동자산         73798549000000
                        이익잉여금       178284102000000
                        자본금            897514000000
                        자본총계        183316724000000
                        자산총계        229664427000000
                 IS     당기순이익        15615018000000
                        매출액         166311191000000
                        법인세차감전 순이익   20451923000000
                        영업이익         20518974000000

In [42]:
df_summary["account_nm"].unique()

array(['유동자산', '비유동자산', '자산총계', '유동부채', '비유동부채', '부채총계', '자본금', '이익잉여금',
       '자본총계', '매출액', '영업이익', '법인세차감전 순이익', '당기순이익'], dtype=object)

In [43]:
df_summary.loc[:, ["fs_div", "fs_nm"]].loc[~df_summary.loc[:, ["fs_div", "fs_nm"]].duplicated(keep="first")]

,fs_div,fs_nm
0,CFS,연결재무제표
13,OFS,재무제표


## Query 와 SQL 만들기

- [시간]: BS: '제\*\*기', '20\*\*년도', '올해', '작년' / IS: '20\*\*년도 부터 20\*\*년 까지', 
- [계정]: BS: '유동자산', '비유동자산', '자산총계', '유동부채', '비유동부채', '부채총계', '자본금', '이익잉여금', '자본총계' / IS: '매출액', '영업이익', '법인세차감전 순이익', '당기순이익'
- [질문]: '얼마나돼', '얼마야', '어떻게돼'

**format의 형태**

틀린 질문을 만들수 없게 로직짜기

단순 계정 관련 질문:
- Q: 삼성전자 [시간]의 [계정]은/는 [질문]? 
- Q: [시간]에서 삼성전자의 [계정]은/는 [질문]?

## XBRL 표준계정과목체계(계정과목)

In [44]:
df_bs1 = dart.xbrl_taxonomy(sj_div="BS1")
df_is1 = dart.xbrl_taxonomy(sj_div="IS1")
bs1_dict = {k: v for k, v in df_bs1.loc[:, ["label_kor", "account_nm"]].values}
is1_dict = {k: v for k, v in df_is1.loc[:, ["label_kor", "account_nm"]].values}

In [45]:
for x, v in list(is1_dict.items()):
    if "당기순이익" in x:
        print(x, v)
        break

당기순이익(손실) ProfitLoss


In [46]:
account_dict = {
    "유동자산" : "CurrentAssets",
    "비유동자산" : "NoncurrentAssets",
    "자산총계" : "Assets",
    "유동부채" : "CurrentLiabilities",
    "비유동부채" : "NoncurrentLiabilities",
    "부채총계" : "Liabilities",
    "자본금" : "IssuedCapital",
    "이익잉여금" : "RetainedEarnings",
    "자본총계" : "Equity",
    "매출액" : "Revenue",
    "영업이익" : "OperatingIncomeLoss",
    "법인세차감전 순이익" : "ProfitLossBeforeTax",
    "당기순이익" : "ProfitLoss"
}

In [47]:
for sj, acc in df_summary.loc[df_summary["fs_div"] == "CFS", ["sj_div", "account_nm"]].values:
    print(f"{acc} = {account_dict.get(acc)}")

유동자산 = CurrentAssets
비유동자산 = NoncurrentAssets
자산총계 = Assets
유동부채 = CurrentLiabilities
비유동부채 = NoncurrentLiabilities
부채총계 = Liabilities
자본금 = IssuedCapital
이익잉여금 = RetainedEarnings
자본총계 = Equity
매출액 = Revenue
영업이익 = OperatingIncomeLoss
법인세차감전 순이익 = ProfitLossBeforeTax
당기순이익 = ProfitLoss


In [20]:
def preprocess_finstate(df):
#     account_dict = {
#         "유동자산" : "CurrentAssets",
#         "비유동자산" : "NoncurrentAssets",
#         "자산총계" : "Assets",
#         "유동부채" : "CurrentLiabilities",
#         "비유동부채" : "NoncurrentLiabilities",
#         "부채총계" : "Liabilities",
#         "자본금" : "IssuedCapital",
#         "이익잉여금" : "RetainedEarnings",
#         "자본총계" : "Equity",
#         "매출액" : "Revenue",
#         "영업이익" : "OperatingIncomeLoss",
#         "법인세차감전 순이익" : "ProfitLossBeforeTax",
#         "당기순이익" : "ProfitLoss"
#     }
#     df.loc[:, "account_nm"] = df.loc[:, "account_nm"].apply(account_dict.get)

    for col in ["thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]:
        if col in df.columns:
            df.loc[:, col] = df.loc[:, col].apply(lambda x: int("".join(x.split(","))))
        else:
            
    df.loc[:, "bsns_year"] = df.loc[:, "bsns_year"].astype(int)
    return df

In [49]:
corp = "005930"
bsns_years = list(range(2015, 2021))

In [50]:
df_receipts = pd.concat([preprocess_finstate(dart.finstate(corp, bsns_year, reprt_code="11011")).iloc[:12, :-1] for bsns_year in bsns_years]).reset_index(drop=True)
df_receipts.loc[:, "bsns_year"] = df_receipts.loc[:, "bsns_year"].astype(int)

In [51]:
df_receipts.tail()

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount
67,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,이익잉여금,제 52 기,2020.12.31 현재,271068211000000,제 51 기,2019.12.31 현재,254582894000000,제 50 기,2018.12.31 현재,242698956000000
68,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,자본총계,제 52 기,2020.12.31 현재,275948016000000,제 51 기,2019.12.31 현재,262880421000000,제 50 기,2018.12.31 현재,247753177000000
69,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,IS,손익계산서,매출액,제 52 기,2020.01.01 ~ 2020.12.31,236806988000000,제 51 기,2019.01.01 ~ 2019.12.31,230400881000000,제 50 기,2018.01.01 ~ 2018.12.31,243771415000000
70,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,IS,손익계산서,영업이익,제 52 기,2020.01.01 ~ 2020.12.31,35993876000000,제 51 기,2019.01.01 ~ 2019.12.31,27768509000000,제 50 기,2018.01.01 ~ 2018.12.31,58886669000000
71,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,IS,손익계산서,법인세차감전 순이익,제 52 기,2020.01.01 ~ 2020.12.31,36345117000000,제 51 기,2019.01.01 ~ 2019.12.31,30432189000000,제 50 기,2018.01.01 ~ 2018.12.31,61159958000000


In [27]:
df_receipts.to_sql(crop, conn, index=True)

## receipts all

In [22]:
df

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_amount,bfefrmtrm_amount,ord
0,20180402003237,11011,2017,01263022,282330,CFS,연결재무제표,BS,재무상태표,유동자산,제 1 기,2017.12.31 현재,"566,642,689,601",-,-,1
1,20180402003237,11011,2017,01263022,282330,CFS,연결재무제표,BS,재무상태표,비유동자산,제 1 기,2017.12.31 현재,"710,706,834,487",-,-,3
2,20180402003237,11011,2017,01263022,282330,CFS,연결재무제표,BS,재무상태표,자산총계,제 1 기,2017.12.31 현재,"1,277,349,524,088",-,-,5
3,20180402003237,11011,2017,01263022,282330,CFS,연결재무제표,BS,재무상태표,유동부채,제 1 기,2017.12.31 현재,"681,364,595,901",-,-,7
4,20180402003237,11011,2017,01263022,282330,CFS,연결재무제표,BS,재무상태표,비유동부채,제 1 기,2017.12.31 현재,"207,340,730,652",-,-,9
5,20180402003237,11011,2017,01263022,282330,CFS,연결재무제표,BS,재무상태표,부채총계,제 1 기,2017.12.31 현재,"888,705,326,553",-,-,11
6,20180402003237,11011,2017,01263022,282330,CFS,연결재무제표,BS,재무상태표,자본금,제 1 기,2017.12.31 현재,"17,283,906,000",-,-,13
7,20180402003237,11011,2017,01263022,282330,CFS,연결재무제표,BS,재무상태표,이익잉여금,제 1 기,2017.12.31 현재,"28,683,646,742",-,-,17
8,20180402003237,11011,2017,01263022,282330,CFS,연결재무제표,BS,재무상태표,자본총계,제 1 기,2017.12.31 현재,"388,644,197,535",-,-,21
9,20180402003237,11011,2017,01263022,282330,CFS,연결재무제표,IS,손익계산서,매출액,제 1 기,2017.11.01 ~ 2017.12.31,"938,735,224,712",-,-,23


In [21]:
bsns_years = list(range(2015, 2021))
dfs = []
for corp in tqdm(stocks_syms):
    for bsns_year in bsns_years:
        df = dart.finstate(corp, bsns_year)
        if df is None:
            continue
        else:
            df = preprocess_finstate(df.iloc[:12, :-1])
            dfs.append(df)

df_receipts = pd.concat(dfs)

  0%|          | 0/808 [00:00<?, ?it/s]

ValueError: invalid literal for int() with base 10: '-'

In [15]:
corp = stocks_syms[0]

In [17]:
dart.finstate(corp, 2012)

In [ ]:
df_receipts = pd.DataFrame([dart.finstate(s) for s in tqdm(stocks_syms)]).loc[:, basic_cols]

In [23]:
# save to sql
dfs = [preprocess_finstate(dart.finstate(corp, bsns_year, reprt_code="11011")).iloc[:12] for bsns_year in bsns_years]
cols = ["stock_code", "rcept_no", "bsns_year"]
rcept_nos = pd.DataFrame([df.loc[0, cols] for df in dfs], columns=cols).reset_index(drop=True)
# rcept_nos["table_name"] = rcept_nos["stock_code"] + rcept_nos["rcept_no"]
rcept_nos

,stock_code,rcept_no,bsns_year
0,005930,20160330003536,2015
1,005930,20170331004518,2016
2,005930,20180402005019,2017
3,005930,20190401004781,2018
4,005930,20200330003851,2019
5,005930,20210309000744,2020


In [24]:
# rcept_nos.to_sql("receipts", conn, index=False)
# cols = [
#     'rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'stock_code', 
#     'fs_div', 'fs_nm', 'sj_div', 'sj_nm', 'account_nm', 'thstrm_nm',
#     'thstrm_dt', 'thstrm_amount', 'frmtrm_nm', 'frmtrm_dt', 'frmtrm_amount',
#     'bfefrmtrm_nm', 'bfefrmtrm_dt', 'bfefrmtrm_amount'
# ]
# for (stock_code, rcept_no, bsns_year, table_name), df in zip(rcept_nos.values, dfs):
#     assert df.loc[0, "bsns_year"] == bsns_year, "error"
#     df.to_sql(table_name, conn, index=True)

In [22]:
cols = [
    'rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'stock_code', 
    'fs_div', 'fs_nm', 'sj_div', 'sj_nm', 'account_nm', 'thstrm_nm',
    'thstrm_dt', 'thstrm_amount', 'frmtrm_nm', 'frmtrm_dt', 'frmtrm_amount',
    'bfefrmtrm_nm', 'bfefrmtrm_dt', 'bfefrmtrm_amount'
]
for (stock_code, rcept_no, bsns_year, table_name), df in zip(rcept_nos.values, dfs):
    assert df.loc[0, "bsns_year"] == bsns_year, "error"
    df.to_sql(table_name, conn, index=True)

In [27]:
dfs[0]

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
0,20160330003536,11011,2015,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동자산,제 47 기,2015.12.31 현재,124814725000000,제 46 기,2014.12.31 현재,115146026000000,제 45 기,2013.12.31 현재,110760271000000,1
1,20160330003536,11011,2015,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동자산,제 47 기,2015.12.31 현재,117364796000000,제 46 기,2014.12.31 현재,115276932000000,제 45 기,2013.12.31 현재,103314747000000,3
2,20160330003536,11011,2015,00126380,005930,CFS,연결재무제표,BS,재무상태표,자산총계,제 47 기,2015.12.31 현재,242179521000000,제 46 기,2014.12.31 현재,230422958000000,제 45 기,2013.12.31 현재,214075018000000,5
3,20160330003536,11011,2015,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동부채,제 47 기,2015.12.31 현재,50502909000000,제 46 기,2014.12.31 현재,52013913000000,제 45 기,2013.12.31 현재,51315409000000,7
4,20160330003536,11011,2015,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동부채,제 47 기,2015.12.31 현재,12616807000000,제 46 기,2014.12.31 현재,10320857000000,제 45 기,2013.12.31 현재,12743599000000,9
5,20160330003536,11011,2015,00126380,005930,CFS,연결재무제표,BS,재무상태표,부채총계,제 47 기,2015.12.31 현재,63119716000000,제 46 기,2014.12.31 현재,62334770000000,제 45 기,2013.12.31 현재,64059008000000,11
6,20160330003536,11011,2015,00126380,005930,CFS,연결재무제표,BS,재무상태표,이익잉여금,제 47 기,2015.12.31 현재,185132014000000,제 46 기,2014.12.31 현재,169529604000000,제 45 기,2013.12.31 현재,148600282000000,17
7,20160330003536,11011,2015,00126380,005930,CFS,연결재무제표,BS,재무상태표,자본총계,제 47 기,2015.12.31 현재,179059805000000,제 46 기,2014.12.31 현재,168088188000000,제 45 기,2013.12.31 현재,150016010000000,21
8,20160330003536,11011,2015,00126380,005930,CFS,연결재무제표,IS,손익계산서,매출액,제 47 기,2015.01.01 ~ 2015.12.31,200653482000000,제 46 기,2014.01.01 ~ 2014.12.31,206205987000000,제 45 기,2013.01.01 ~ 2013.12.31,228692667000000,23
9,20160330003536,11011,2015,00126380,005930,CFS,연결재무제표,IS,손익계산서,영업이익,제 47 기,2015.01.01 ~ 2015.12.31,26413442000000,제 46 기,2014.01.01 ~ 2014.12.31,25025071000000,제 45 기,2013.01.01 ~ 2013.12.31,36785013000000,25


In [28]:
dfs[1]

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
0,20170331004518,11011,2016,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동자산,제 48 기,2016.12.31 현재,141429704000000,제 47 기,2015.12.31 현재,124814725000000,제 46 기,2014.12.31 현재,115146026000000,1
1,20170331004518,11011,2016,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동자산,제 48 기,2016.12.31 현재,120744620000000,제 47 기,2015.12.31 현재,117364796000000,제 46 기,2014.12.31 현재,115276932000000,3
2,20170331004518,11011,2016,00126380,005930,CFS,연결재무제표,BS,재무상태표,자산총계,제 48 기,2016.12.31 현재,262174324000000,제 47 기,2015.12.31 현재,242179521000000,제 46 기,2014.12.31 현재,230422958000000,5
3,20170331004518,11011,2016,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동부채,제 48 기,2016.12.31 현재,54704095000000,제 47 기,2015.12.31 현재,50502909000000,제 46 기,2014.12.31 현재,52013913000000,7
4,20170331004518,11011,2016,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동부채,제 48 기,2016.12.31 현재,14507196000000,제 47 기,2015.12.31 현재,12616807000000,제 46 기,2014.12.31 현재,10320857000000,9
5,20170331004518,11011,2016,00126380,005930,CFS,연결재무제표,BS,재무상태표,부채총계,제 48 기,2016.12.31 현재,69211291000000,제 47 기,2015.12.31 현재,63119716000000,제 46 기,2014.12.31 현재,62334770000000,11
6,20170331004518,11011,2016,00126380,005930,CFS,연결재무제표,BS,재무상태표,이익잉여금,제 48 기,2016.12.31 현재,193086317000000,제 47 기,2015.12.31 현재,185132014000000,제 46 기,2014.12.31 현재,169529604000000,17
7,20170331004518,11011,2016,00126380,005930,CFS,연결재무제표,BS,재무상태표,자본총계,제 48 기,2016.12.31 현재,192963033000000,제 47 기,2015.12.31 현재,179059805000000,제 46 기,2014.12.31 현재,168088188000000,21
8,20170331004518,11011,2016,00126380,005930,CFS,연결재무제표,IS,손익계산서,매출액,제 48 기,2016.01.01 ~ 2016.12.31,201866745000000,제 47 기,2015.01.01 ~ 2015.12.31,200653482000000,제 46 기,2014.01.01 ~ 2014.12.31,206205987000000,23
9,20170331004518,11011,2016,00126380,005930,CFS,연결재무제표,IS,손익계산서,영업이익,제 48 기,2016.01.01 ~ 2016.12.31,29240672000000,제 47 기,2015.01.01 ~ 2015.12.31,26413442000000,제 46 기,2014.01.01 ~ 2014.12.31,25025071000000,25


In [26]:
pd.concat(dfs)

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
0,20160330003536,11011,2015,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동자산,제 47 기,2015.12.31 현재,124814725000000,제 46 기,2014.12.31 현재,115146026000000,제 45 기,2013.12.31 현재,110760271000000,1
1,20160330003536,11011,2015,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동자산,제 47 기,2015.12.31 현재,117364796000000,제 46 기,2014.12.31 현재,115276932000000,제 45 기,2013.12.31 현재,103314747000000,3
2,20160330003536,11011,2015,00126380,005930,CFS,연결재무제표,BS,재무상태표,자산총계,제 47 기,2015.12.31 현재,242179521000000,제 46 기,2014.12.31 현재,230422958000000,제 45 기,2013.12.31 현재,214075018000000,5
3,20160330003536,11011,2015,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동부채,제 47 기,2015.12.31 현재,50502909000000,제 46 기,2014.12.31 현재,52013913000000,제 45 기,2013.12.31 현재,51315409000000,7
4,20160330003536,11011,2015,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동부채,제 47 기,2015.12.31 현재,12616807000000,제 46 기,2014.12.31 현재,10320857000000,제 45 기,2013.12.31 현재,12743599000000,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,이익잉여금,제 52 기,2020.12.31 현재,271068211000000,제 51 기,2019.12.31 현재,254582894000000,제 50 기,2018.12.31 현재,242698956000000,17
8,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,BS,재무상태표,자본총계,제 52 기,2020.12.31 현재,275948016000000,제 51 기,2019.12.31 현재,262880421000000,제 50 기,2018.12.31 현재,247753177000000,21
9,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,IS,손익계산서,매출액,제 52 기,2020.01.01 ~ 2020.12.31,236806988000000,제 51 기,2019.01.01 ~ 2019.12.31,230400881000000,제 50 기,2018.01.01 ~ 2018.12.31,243771415000000,23
10,20210309000744,11011,2020,00126380,005930,CFS,연결재무제표,IS,손익계산서,영업이익,제 52 기,2020.01.01 ~ 2020.12.31,35993876000000,제 51 기,2019.01.01 ~ 2019.12.31,27768509000000,제 50 기,2018.01.01 ~ 2018.12.31,58886669000000,25


In [23]:
res = c.execute("SELECT name FROM sqlite_master WHERE type='table';")
res.fetchall()

[('kospi',),
 ('company',),
 ('receipts',),
 ('00593020160330003536',),
 ('00593020170331004518',),
 ('00593020180402005019',),
 ('00593020190401004781',),
 ('00593020200330003851',),
 ('00593020210309000744',)]

---


## Finstate_all

sj_nm, sj_div
- 재무상태표(BS): Balance Sheet
- 포괄손익계산서(CIS): Comprehensive Income Statement
- 현금흐름표(CF): Cash Flow statement
- 자본변동표(SCE): Statement of Changes in Equity

In [16]:
df_state = dart.finstate_all(corp, bsns_year, reprt_code="11011", fs_div="CFS")
df_state["sj_nm"].unique()

array(['재무상태표', '손익계산서', '포괄손익계산서', '현금흐름표', '자본변동표'], dtype=object)

In [17]:
df_state.head()

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,thstrm_add_amount
0,20210309000744,11011,2020,00126380,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 52 기,198215579000000,제 51 기,181385260000000,제 50 기,174697424000000,1,NaN
1,20210309000744,11011,2020,00126380,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 52 기,29382578000000,제 51 기,26885999000000,제 50 기,30340505000000,2,NaN
2,20210309000744,11011,2020,00126380,BS,재무상태표,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,-,제 52 기,92441703000000,제 51 기,76252052000000,제 50 기,65893797000000,3,NaN
3,20210309000744,11011,2020,00126380,BS,재무상태표,-표준계정코드 미사용-,단기상각후원가금융자산,-,제 52 기,2757111000000,제 51 기,3914216000000,제 50 기,2703693000000,4,NaN
4,20210309000744,11011,2020,00126380,BS,재무상태표,ifrs-full_CurrentFinancialAssetsAtFairValueThr...,단기당기손익-공정가치금융자산,-,제 52 기,71451000000,제 51 기,1727436000000,제 50 기,2001948000000,5,NaN


In [61]:
df_state.keys()

Index(['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'sj_div', 'sj_nm',
       'account_id', 'account_nm', 'account_detail', 'thstrm_nm',
       'thstrm_amount', 'frmtrm_nm', 'frmtrm_amount', 'bfefrmtrm_nm',
       'bfefrmtrm_amount', 'ord', 'thstrm_add_amount'],
      dtype='object')

In [62]:
df_state['sj_div'].unique()

array(['BS', 'IS', 'CIS', 'CF', 'SCE'], dtype=object)

In [63]:
for sj in df_state['sj_div'].unique():
    df_report = df_state.loc[df_state['sj_div'] == 'CIS']
    break

In [42]:
df_state['account_nm'].unique()

array(['유동자산', '현금및현금성자산', '단기금융상품', '단기투자자산', '매출채권', '기타수취채권', '재고자산',
       '당기법인세자산', '기타유동자산', '기타금융자산', '비유동자산', '종속기업, 관계기업 및 공동기업투자',
       '장기매출채권', '장기투자자산', '유형자산', '사용권자산', '무형자산', '투자부동산', '이연법인세자산',
       '종업원급여자산', '기타비유동자산', '자산총계', '유동부채', '매입채무', '미지급금', '기타지급채무',
       '차입금', '충당부채', '당기법인세부채', '리스부채', '기타유동부채', '기타금융부채', '비유동부채',
       '장기미지급금', '확정급여부채', '이연법인세부채', '기타비유동부채', '부채총계', '지배기업의 소유지분',
       '자본금', '자본잉여금', '기타자본', '기타포괄손익누계액', '이익잉여금', '비지배지분', '자본총계',
       '부채및자본총계', '매출액', '매출원가', '매출총이익', '판매비와관리비', '영업이익', '금융수익',
       '금융비용', '지분법투자 관련 손익', '기타영업외수익', '기타영업외비용', '법인세비용차감전순이익',
       '법인세비용', '당기순이익', '법인세차감후 기타포괄손익', '확정급여제도의 재측정요소', '해외사업장환산외환차이',
       '파생상품평가손익', '관계기업의 기타포괄손익에 대한 지분', '총포괄손익', '지배기업의 소유주지분',
       '기본주당순이익', '희석주당순이익', '영업활동 현금흐름', '영업으로부터 창출된 현금흐름', '이자의 수취',
       '이자의 지급', '배당금의 수취', '법인세의 납부', '투자활동 현금흐름', '단기금융상품의 감소',
       '단기금융상품의 증가', '단기투자자산의 순증감', '기타금융자산의 감소', '기타금융자산의 증가',
       '기타수취채권의 감소', '기타수취

## Major shareholders

- corp (문자열): 검색대상 회사의 종목코드를 지정합니다. 고유번호, 회사이름도 가능합니다.

반환값 (DataFrame): 조회 결과를 데이터프레임(DataFrame)으로 반환합니다. 데이터프레임의 각 컬럼은 다음과 같습니다.

- rcept_no: 접수번호
- rcept_dt: 접수일자
- corp_code: 종목코드
- corp_name: 회사명
- report_tp: 보고구분
- repror: 대표보고자
- stkqy: 보유주식등의 수
- stkqy_irds: 보유주식등의 증감
- stkrt: 보유비율
- stkrt_irds: 보유비율 증감
- ctr_stkqy: 주요체결 주식등의 수
- ctr_stkrt: 주요체결 보유비율
- report_resn: 보고사유

In [ ]:
dart.